In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.stattools import adfuller
import warnings

warnings.filterwarnings("ignore")  # Ignorar advertencias

In [ ]:
# Subir el archivo desde el sistema local
from google.colab import files
uploaded = files.upload()  # Abre un cuadro de diálogo para subir archivos

# Leer el archivo CSV en un DataFrame
df = pd.read_csv('Enso.csv')  # Asegúrate de que el nombre del archivo coincida

Saving Enso.csv to Enso.csv


In [ ]:
# Convertir la columna de fecha en un objeto de fecha (si es necesario)
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Establecer la columna de fecha como índice si está presente
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)


# Visualizar las primeras filas del DataFrame para entender su estructura
print("Primeras filas del DataFrame:")
print(df.head())

Primeras filas del DataFrame:
            nino12  nino3  nino4  nino34
Date                                    
1870-01-01   -1.50  -1.35  -0.48   -1.00
1870-02-01   -0.96  -1.20  -1.16   -1.20
1870-03-01   -0.65  -0.76  -1.01   -0.83
1870-04-01   -0.32  -0.78  -1.00   -0.81
1870-05-01   -0.64  -1.15  -1.08   -1.27


In [ ]:
# Definir una función para ajustar un modelo Gradient Boosting y devolver los estadísticos de predicción
def gradient_boosting_forecast(series, n_estimators=100, learning_rate=0.1, max_depth=3):
    # Asegurarse de que la serie sea un vector unidimensional (sin nulos)
    series = series.dropna().astype(float)

    # Dividir los datos en conjunto de entrenamiento (70%) y prueba (30%)
    train, test = train_test_split(series, test_size=0.3, shuffle=False)

    # Crear una variable de tiempo para usar como la variable independiente
    X_train = np.arange(len(train)).reshape(-1, 1)
    X_test = np.arange(len(train), len(train) + len(test)).reshape(-1, 1)

    # Ajustar el modelo Gradient Boosting
    model = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    model.fit(X_train, train)

    # Generar pronósticos
    forecast = model.predict(X_test)

    # Calcular métricas de evaluación
    mse = mean_squared_error(test, forecast)  # Error Cuadrático Medio
    rmse = np.sqrt(mse)  # Raíz del Error Cuadrático Medio
    mae = mean_absolute_error(test, forecast)  # Error Absoluto Medio
    r2 = r2_score(test, forecast)  # Coeficiente de Determinación

    # Calcular MAPE manejando valores cero
    mape = np.mean(np.abs((test - forecast) / (test + np.finfo(float).eps))) * 100  # Evitar división por cero

    # Realizar la prueba de Dickey-Fuller aumentada para la estacionariedad
    adf_test = adfuller(series)
    adf_statistic = adf_test[0]  # Estadístico ADF
    adf_p_value = adf_test[1]  # Valor p de la prueba ADF

    # Devolver las métricas y los estadísticos en un diccionario
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'adf_statistic': adf_statistic,
        'adf_p_value': adf_p_value
    }

# Aplicar el modelo Gradient Boosting a cada una de las series de tiempo numéricas en el DataFrame
gb_results = {}
for column in df.select_dtypes(include=[np.number]).columns:
    print(f"Procesando la serie de tiempo: {column}")
    gb_results[column] = gradient_boosting_forecast(df[column])

# Crear un DataFrame para almacenar los resultados
results_df = pd.DataFrame(gb_results).T  # Transponer para una mejor presentación
results_df.columns = ['MSE', 'RMSE', 'MAE', 'R²', 'MAPE', 'ADF Statistic', 'ADF P-value']  # Renombrar columnas

# Mostrar el DataFrame con los resultados
print("Resumen de los resultados de Gradient Boosting:")
print(results_df)

# Guardar el DataFrame en un archivo CSV
results_df.to_csv('gradient_boosting_model_comparison.csv', index=True)  # Guardar el DataFrame en un archivo CSV
files.download('gradient_boosting_model_comparison.csv')  # Descargar el archivo CSV generado

Procesando la serie de tiempo: nino12
Procesando la serie de tiempo: nino3
Procesando la serie de tiempo: nino4
Procesando la serie de tiempo: nino34
Resumen de los resultados de Gradient Boosting:
             MSE      RMSE       MAE        R²          MAPE  ADF Statistic  \
nino12  1.177186  1.084982  0.733268 -0.044966  7.202489e+14     -11.532023   
nino3   0.933971  0.966422  0.788832 -0.159192  1.562997e+15     -10.225140   
nino4   0.547843  0.740164  0.578101 -0.384421  7.082441e+14      -9.779188   
nino34  1.146157  1.070587  0.896904 -0.536373  1.063192e+15     -10.125987   

         ADF P-value  
nino12  3.811493e-21  
nino3   5.200632e-18  
nino4   6.805263e-17  
nino34  9.179166e-18  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>